In [21]:
from IPython.core.display import display, HTML # for some notebook formatting.
import mlrose_hiive
import numpy as np
import logging
import networkx as nx
import matplotlib.pyplot as plt
import string

from ast import literal_eval

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.metrics import accuracy_score
from mlrose_hiive import QueensGenerator, MaxKColorGenerator, TSPGenerator
from mlrose_hiive import SARunner, GARunner, NNGSRunner

from mlrose_hiive.opt_probs.discrete_opt import DiscreteOpt

# switch off the chatter
logging.basicConfig(level=logging.WARNING)

In [22]:
class GraphGenusFitness(CustomFitness):
    def __init__(self, adjacency_list):
        self.adjacency_list = adjacency_list

    def evaluate(self, state):
        orderedPairs = [(state[i], state[j]) for i in range(len(state)) for j in self.adjacency_list[state[i]] if state[j] > state[i]]
        genus = self.calculate_genus(orderedPairs)
        return genus

    def calculate_genus(self, orderedPairs):
        return self.euler_formula(orderedPairs, self.adjacency_list)

    def euler_formula(self, orderedPairs, adj_list):
        V = len(adj_list)
        E = sum(len(neighbors) for neighbors in adj_list.values()) / 2
        F = countFaces(orderedPairs.copy(), adj_list)
        genus = int((2 - (V - E + F)) / 2)
        return genus

def countFaces(ord_pairs, adj_list):
    faces = []
    while ord_pairs:
        face = [ord_pairs[0]]
        ord_pairs.pop(0)
        i = face[0][1]
        while True:
            possible_next_edges = [pair for pair in ord_pairs if pair[0] == i]
            if not possible_next_edges:
                break
            next_edge = min(possible_next_edges, key=lambda x: adj_list[i].index(x[1]))
            if next_edge == face[0]:
                break
            face.append(next_edge)
            ord_pairs.remove(next_edge)
            i = next_edge[1]
        faces.append(face)
    return len(faces)


In [23]:
class GraphGenusOpt(DiscreteOpt):
    def __init__(self, length=None, fitness_fn=None, maximize=False, max_val=2, adjacency_list=None):
        if (fitness_fn is None) and (adjacency_list is None):
            raise Exception("At least one of fitness_fn and adjacency_list must be specified.")
        elif fitness_fn is None:
            fitness_fn = GraphGenusFitness(adjacency_list)
        self.adj_list = adjacency_list
        if length is None:
            if adjacency_list is not None:
                length = len(adjacency_list)
        self.length = length
        super().__init__(length=length, fitness_fn=fitness_fn, maximize=maximize, max_val=max_val)
        
        if self.fitness_fn.get_prob_type() != 'discrete':
            raise Exception("fitness_fn must have problem type 'discrete'.")
            
        self.prob_type = 'graph_genus'

    def reset(self):
        self.state = self.random()
        self.fitness = self.eval_fitness(self.state)
        self.fevals = {}
        self.fitness_evaluations = 0
        self.current_iteration = 0

    def random(self):
        return np.random.randint(0, 2, self.length)

    def random_neighbor(self, state):
        neighbor = state.copy()
        vertex = np.random.randint(len(state))
        neighbor[vertex] = 1 - neighbor[vertex]  # Flip the bit
        return neighbor

    def find_neighbors(self, state):
        neighbors = []
        for i in range(len(state)):
            neighbor = state.copy()
            neighbor[i] = 1 - neighbor[i]  # Flip the bit
            neighbors.append(neighbor)
        return neighbors


In [24]:
class GraphGenusGenerator:
    @staticmethod
    def generate(seed, number_of_nodes=20, max_connections_per_node=4, complete_graph=False):
        np.random.seed(seed)
        
        if complete_graph:
            # Generate a complete graph
            g = nx.complete_graph(number_of_nodes)
        else:
            # All nodes have to be connected, somehow.
            node_connection_counts = 1 + np.random.randint(max_connections_per_node, size=number_of_nodes)

            node_connections = {}
            nodes = range(number_of_nodes)
            for n in nodes:
                all_other_valid_nodes = [o for o in nodes if (o != n and (o not in node_connections or
                                                                          n not in node_connections[o]))]
                count = min(node_connection_counts[n], len(all_other_valid_nodes))
                other_nodes = sorted(np.random.choice(all_other_valid_nodes, count, replace=False))
                node_connections[n] = [(n, o) for o in other_nodes]

            # Check connectivity
            g = nx.Graph()
            g.add_edges_from([x for y in node_connections.values() for x in y])

            for n in nodes:
                cannot_reach = [(n, o) if n < o else (o, n) for o in nodes if o not in nx.bfs_tree(g, n).nodes()]
                for s, f in cannot_reach:
                    g.add_edge(s, f)
                    check_reach = len([(n, o) if n < o else (o, n) for o in nodes if o not in nx.bfs_tree(g, n).nodes()])
                    if check_reach == 0:
                        break

        adjacency_list = {node: list(g.neighbors(node)) for node in g.nodes()}
        problem = GraphGenusOpt(length=number_of_nodes, fitness_fn=GraphGenusFitness(adjacency_list=adjacency_list), maximize=False, adjacency_list=g)
        return problem


In [25]:
import matplotlib.pyplot as plt
from mlrose_hiive import hill_climb

# Generate a new Graph Genus problem using a fixed seed.
problem = GraphGenusGenerator().generate(seed=123458, number_of_nodes=4, max_connections_per_node=3, complete_graph=True)

# Run the hill climbing algorithm to solve the problem
best_state, best_fitness = hill_climb(problem, max_iters=100, restarts=10, curve=False, random_state=123)

# Print the result
print("Best State:", best_state)
print("Best Fitness (Genus):", best_fitness)

# Get the adjacency list (which is a graph object)
graph = problem.adj_list

# Draw the generated graph
nx.draw(graph, with_labels=True, node_color='lightblue', node_size=500, font_size=10, font_weight='bold')
plt.title("Input Graph for Graph Genus Problem")
plt.show()


AttributeError: 'GraphGenusFitness' object has no attribute 'problem_type'